This file compute demand parameter using IV regression rather than OLS. Because OLS is biased. 


Takeaway from demand parameters: 

Consumers are willing to pay 2 dollars more to upgrate to Entire home from private room.

Consumers are willing to pay 71 dolars more to book a propertity mangaged by super hosts.

The average profit margin for the host is 8% (rate of return) 


In [2]:
# estimate market demand, price elasticites, markup

import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
import statsmodels.api as sm
import seaborn as sns
import matplotlib.dates as mdates
from datetime import date
from sklearn.model_selection import train_test_split
import math




In [3]:
df = pd.read_csv('LA\March\listings.csv')

In [5]:
#data cleaning. Price data has $ sign. The following code extracts the number
df['price'] = df['price'].replace({r'\$':''},regex = True).replace({r',':''},regex = True).astype('float')
df.price
# delete the listings which are more expensive than $800
df1=df[df.price<800]
df1.reviews_per_month.isna().sum()/len(df1)

#get the subset of the data without missing 

df2= df1[~df1.beds.isna()&~df1.reviews_per_month.isna()&~df1.review_scores_accuracy.isna()&~df1.review_scores_checkin.isna()
         &~df1.review_scores_communication.isna()&~df1.review_scores_location.isna()&~df1.review_scores_value.isna()]
len(df2)/len(df1)

0.720346127144699

In [106]:
# for top 30 neighbourhood market # this can be replaced by hotspot

listtop30 = df2.neighbourhood_cleansed.value_counts().head(30).index.tolist()
df3=df2[df2["neighbourhood_cleansed"].isin(listtop30)]



In [108]:
df3 = df3.copy() 
# create market share from reviews
df3.loc[:,'share'] =df3['reviews_per_month']/(df3.groupby('neighbourhood_cleansed')['reviews_per_month'].transform('sum')*3)

# 2/3 is arbitrary, won't affect result much, create mean valuation of the listing
df3['mean_value'] =np.log(df3['share']/(2/3))

The following block run simple OLS.  

In [109]:
# for indirect utility
Y=df3.mean_value
#with amenties


license_dummy=1*(~df3.license.isna()) 
# Fill NAN by "missing" string
df3[['host_response_time']] = df3[['host_response_time']].fillna('Missing_response')
host_response_dummy=pd.get_dummies(df3.host_response_time)
#room_type dummy
room_type_dummy=pd.get_dummies(df3.room_type)
super_host_dummy =1*(df3.host_is_superhost=="t")

amentity_count=df3.amenities.str.split().str.len()
wifi = 1*df3.amenities.str.lower().str.contains('wifi', regex=True) 
washer = 1*df3.amenities.str.lower().str.contains('washer', regex=True) 
dryer = 1*df3.amenities.str.lower().str.replace("hair dryer", " ").str.lower().str.contains('dryer', regex=True)
hair_dryer = 1*df3.amenities.str.lower().str.contains('hair dryer', regex=True) 
free_parking = 1*df3.amenities.str.lower().str.contains('free parking', regex=True) 
AC = 1*df3.amenities.str.lower().str.contains('air conditioning', regex=True) 
TV= 1*df3.amenities.str.lower().str.contains('tv', regex=True) 
hottub = 1*df3.amenities.str.lower().str.contains('hot tub', regex=True) 
coffee = 1*df3.amenities.str.lower().str.contains('coffee', regex=True) 
microwave = 1*df3.amenities.str.lower().str.contains('microwave', regex=True) 
pool = 1*df3.amenities.str.lower().str.contains('pool', regex=True) 
shampoo = 1*df3.amenities.str.lower().str.contains('shampoo', regex=True) 

X=pd.concat([df3.price,df3.beds,license_dummy,
             room_type_dummy,super_host_dummy,df3.review_scores_rating,amentity_count,wifi,
             washer,dryer,hair_dryer,free_parking,AC,TV,hottub,coffee,microwave,pool,shampoo],axis=1)
             
             #df2.host_response_rate,df2.host_acceptance_rate,df2.host_is_superhost,df2.room_type],axis=1)
X=sm.add_constant(X)
mod = sm.OLS(Y,X)
est = mod.fit()

print(est.summary())

                            OLS Regression Results                            
Dep. Variable:             mean_value   R-squared:                       0.210
Model:                            OLS   Adj. R-squared:                  0.209
Method:                 Least Squares   F-statistic:                     190.7
Date:                Mon, 29 May 2023   Prob (F-statistic):               0.00
Time:                        21:57:27   Log-Likelihood:                -25831.
No. Observations:               15052   AIC:                         5.171e+04
Df Residuals:                   15030   BIC:                         5.187e+04
Df Model:                          21                                         
Covariance Type:            nonrobust                                         
                           coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------
const                   -7.5179 

In [110]:
print((est.params['Entire home/apt']-est.params['Private room'])/est.params['price'])
print(est.params['host_is_superhost']/est.params['price'])

-141.69389892182798
-552.5969886000523


Consumers are willing to pay 142 dollars more to upgrate to Entire home from private room.

Consumers are willing to pay 552 dolars more to book a propertity mangaged by super hosts.

This doesn't really make sense.

Because of price endogeneity here, I use instrument variable (IV) to get a correct price coeffcient. 

The following code computes IV results. 


In [112]:
from statsmodels.sandbox.regression.gmm import IV2SLS
# instrument for price endogeneity
df3.loc[:,'IV'] = df3.groupby(['neighbourhood_cleansed'])['id'].transform('count')

Y=df3.mean_value
X=pd.concat([df3.price,df3.beds,license_dummy,
             room_type_dummy,super_host_dummy,df3.review_scores_rating,amentity_count,wifi,
             washer,dryer,hair_dryer,free_parking,AC,TV,hottub,coffee,microwave,pool,shampoo],axis=1)
exog_constant=X
#sm.add_constant(X)
endog = Y
instr_constant = sm.add_constant(pd.concat([df3.IV,df3.beds,license_dummy,
             room_type_dummy,super_host_dummy,df3.review_scores_rating,amentity_count,wifi,
             washer,dryer,hair_dryer,free_parking,AC,TV,hottub,coffee,microwave,pool,shampoo],axis=1))
endog_results = IV2SLS(endog, exog_constant, instrument = instr_constant).fit()
endog_results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                          IV2SLS Regression Results                           
==============================================================================
Dep. Variable:             mean_value   R-squared:                     -46.063
Model:                         IV2SLS   Adj. R-squared:                -46.129
Method:                     Two Stage   F-statistic:                       nan
                        Least Squares   Prob (F-statistic):                nan
Date:                Mon, 29 May 2023                                         
Time:                        21:58:15                                         
No. Observations:               15052                                         
Df Residuals:                   15030                                         
Df Model:                          21                                         
========================================================================================
                           coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------
price                   -0.1038      0.016     -6.577      0.000      -0.135      -0.073
beds                     4.2210      0.634      6.662      0.000       2.979       5.463
license                  4.5189      0.677      6.679      0.000       3.193       5.845
Entire home/apt         -8.5537      1.171     -7.305      0.000     -10.849      -6.259
Hotel room             -10.4800      1.825     -5.742      0.000     -14.057      -6.903
Private room           -15.9352      1.561    -10.207      0.000     -18.995     -12.875
Shared room            -20.2491      2.235     -9.061      0.000     -24.630     -15.869
host_is_superhost        0.1972      0.194      1.017      0.309      -0.183       0.577
review_scores_rating     0.9700      0.240      4.040      0.000       0.499       1.441
amenities                0.0538      0.008      6.795      0.000       0.038       0.069
amenities                0.7586      0.820      0.926      0.355      -0.848       2.365
amenities                1.5064      0.436      3.454      0.001       0.652       2.361
amenities               -1.0006      0.317     -3.157      0.002      -1.622      -0.379
amenities                0.0909      0.262      0.347      0.728      -0.422       0.604
amenities                1.1493      0.241      4.761      0.000       0.676       1.622
amenities                2.1472      0.347      6.183      0.000       1.467       2.828
amenities                0.3073      0.315      0.975      0.330      -0.310       0.925
amenities                2.1924      0.440      4.982      0.000       1.330       3.055
amenities                0.7031      0.268      2.623      0.009       0.178       1.228
amenities               -3.4875      0.628     -5.557      0.000      -4.718      -2.257
amenities                0.3017      0.261      1.158      0.247      -0.209       0.812
amenities                0.9005      0.268      3.359      0.001       0.375       1.426
==============================================================================
Omnibus:                     5036.980   Durbin-Watson:                   1.938
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            22806.090
Skew:                           1.579   Prob(JB):                         0.00
Kurtosis:                       8.137   Cond. No.                     7.03e+03
==============================================================================
"""

In [113]:
print(endog_results.params['host_is_superhost']/endog_results.params['price'])
print((endog_results.params['Entire home/apt']-endog_results.params['Private room'])/endog_results.params['price'])

-1.9002046952246943
-71.11866549570298


Consumers are willing to pay 71 dollars more to upgrate to Entire home from private room.

Consumers are willing to pay 2 dolars more to book a propertity mangaged by super hosts.

The following code computes markup based on IV regression results. 

In [114]:
#markup 
markup =[]
df3 = df3.copy()
# number of the markets (hotspots)
M = df3['neighbourhood_cleansed'].nunique()
Group_market = df3.groupby('neighbourhood_cleansed')
#elaciticies
ela = [] 
for num, i in enumerate(df3['neighbourhood_cleansed'].unique()):
    submarket = Group_market.get_group(i)
    share_m = np.array(submarket.share)
    dsdp_m =  endog_results.params['price']*(np.diag(share_m)-np.outer(share_m,share_m)) #price derivatives
    
    #ela
    ela_m =dsdp_m*np.array(np.array(submarket.price).reshape(-1, 1) )/np.array(submarket.share)# that's how i compute elasticities  
    ela.append(ela_m)
    
    # markup
    host_id = submarket.host_id.to_numpy()
    T = host_id [:, np.newaxis] == host_id 
    markup.append(-np.dot(np.linalg.inv(T*dsdp_m),share_m))
     # assign markup back to dataFrame df3
    df3.loc[df3['neighbourhood_cleansed'] == i, 'markup'] = markup[num]

In [ ]:
# for a quick check 
#df3['neighbourhood_cleansed'].unique()
#i ="Rowland Heights"
#submarket = Group_market.get_group(i)
#share_m = np.array(submarket.share)

In [115]:
df3.markup.head() #markup for the first 10 listing in the first market

#profit margain in percentage

df3['margin_perc'] = df3.markup/df3.price
print("average markup per listing is $",str(round(df3.markup.mean(),2)) )
print("average rate of return is",str(round(df3.margin_perc.mean(),2)))
df3['mc'] = df3['price']-df3['markup']

average markup per listing is $ 9.66
average rate of return is 0.08


In [116]:
#np.diag(ela[0]) # own ric elaciticites in the first market 

The following code tries lasso. No siginificant gain in accuracy. 


In [20]:
# Lasso 

from sklearn import datasets
from sklearn.linear_model import LassoCV
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import Lasso

# Define parameter grid
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state=777)
param_grid = {'alpha': [0.0001,0.001, 0.01, 0.1, 1, 10]}
 
# Perform grid search with cross-validation
lasso_cv = GridSearchCV(Lasso(), param_grid, cv=5)
lasso_cv.fit(X_train, Y_train)
 
# Print best parameter values and score
print("Best Parameters:", lasso_cv.best_params_)
print("Best Score:", lasso_cv.best_score_)




Best Parameters: {'alpha': 0.001}
Best Score: 0.20292170807155507


In [21]:
 lasso_cv.cv_results_

{'mean_fit_time': array([0.01189117, 0.01068273, 0.00935316, 0.0062696 , 0.00783801,
        0.01254196]),
 'std_fit_time': array([0.0060644 , 0.00640214, 0.00763692, 0.00767866, 0.00504703,
        0.00592282]),
 'mean_score_time': array([0.0031219 , 0.00190339, 0.0031249 , 0.00624113, 0.00558414,
        0.00160294]),
 'std_score_time': array([0.0062438 , 0.00380678, 0.00624981, 0.0076438 , 0.00233563,
        0.0020585 ]),
 'param_alpha': masked_array(data=[0.0001, 0.001, 0.01, 0.1, 1, 10],
              mask=[False, False, False, False, False, False],
        fill_value='?',
             dtype=object),
 'params': [{'alpha': 0.0001},
  {'alpha': 0.001},
  {'alpha': 0.01},
  {'alpha': 0.1},
  {'alpha': 1},
  {'alpha': 10}],
 'split0_test_score': array([0.22275146, 0.22260922, 0.21416753, 0.11606427, 0.09674889,
        0.06320693]),
 'split1_test_score': array([0.21070995, 0.21105985, 0.20521431, 0.09115998, 0.07329301,
        0.05708773]),
 'split2_test_score': array([0.18417126, 0

In [23]:
lasso = Lasso(alpha=0.0001)
lasso.fit(X_train, Y_train)
Y_pred = lasso.predict(X_test)
MSE = ((Y_pred-Y_test)**2).mean()
print(MSE)
coefs = lasso.coef_

1.8157141626183637


In [24]:
#OLS 

mod = sm.OLS(Y_train,X_train)
est = mod.fit()
Y_pred_OLS=est.predict(X_test)
MSE_OLS = ((Y_pred_OLS-Y_test)**2).mean()
print(MSE_OLS)

1.8155887528108074
